# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3a3499fa90>
├── 'a' --> tensor([[-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379]])
└── 'x' --> <FastTreeValue 0x7f3a3499fa60>
    └── 'c' --> tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                        [-0.6017,  1.1726, -1.1364,  0.7534],
                        [ 0.8182, -1.7573,  0.1515, -0.9661]])

In [4]:
t.a

tensor([[-0.3139, -0.9896,  0.4035],
        [ 0.4778,  1.5925, -0.5379]])

In [5]:
%timeit t.a

67.5 ns ± 0.0286 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3a3499fa90>
├── 'a' --> tensor([[ 0.4759, -0.4802,  1.1278],
│                   [-1.3560, -0.4022, -0.6245]])
└── 'x' --> <FastTreeValue 0x7f3a3499fa60>
    └── 'c' --> tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                        [-0.6017,  1.1726, -1.1364,  0.7534],
                        [ 0.8182, -1.7573,  0.1515, -0.9661]])

In [7]:
%timeit t.a = new_value

70.1 ns ± 0.0189 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3139, -0.9896,  0.4035],
               [ 0.4778,  1.5925, -0.5379]]),
    x: Batch(
           c: tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661]]),
       ),
)

In [10]:
b.a

tensor([[-0.3139, -0.9896,  0.4035],
        [ 0.4778,  1.5925, -0.5379]])

In [11]:
%timeit b.a

61.3 ns ± 0.0288 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0198, -0.3467,  0.2654],
               [ 0.0076, -1.1334, -0.3651]]),
    x: Batch(
           c: tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.0708 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

722 ns ± 0.0852 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 38 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 873 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f39868e8e80>
├── 'a' --> tensor([[[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]],
│           
│                   [[-0.3139, -0.9896,  0.4035],
│                    [ 0.4778,  1.5925, -0.5379]]])
└── 'x' --> <FastTreeValue 0x7f3a34997490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f39febd91f0>
├── 'a' --> tensor([[-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379],
│                   [-0.3139, -0.9896,  0.4035],
│                   [ 0.4778,  1.5925, -0.5379]])
└── 'x' --> <FastTreeValue 0x7f397fc55fa0>
    └── 'c' --> tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                        [-0.6017,  1.1726, -1.1364,  0.7534],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 52.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1137,  0.2588,  0.5976, -0.8652],
                       [-0.6017,  1.1726, -1.1364,  0.7534],
                       [ 0.8182, -1.7573,  0.1515, -0.9661]],
              
                      [[ 1.1137,  0.2588,  0.5976, -0.8652],
                       [-0.6017,  1.1726, -1.1364,  0.7534],
                       [ 0.8182, -1.7573,  0.1515, -0.9661]],
              
                      [[ 1.1137,  0.2588,  0.5976, -0.8652],
                       [-0.6017,  1.1726, -1.1364,  0.7534],
                       [ 0.8182, -1.7573,  0.1515, -0.9661]],
              
                      [[ 1.1137,  0.2588,  0.5976, -0.8652],
                       [-0.6017,  1.1726, -1.1364,  0.7534],
                       [ 0.8182, -1.7573,  0.1515, -0.9661]],
              
                      [[ 1.1137,  0.2588,  0.5976, -0.8652],
                       [-0.6017,  1.1726, -1.1364,  0.7534],
                       [ 0.8182, -1.7573,  0.1515, -0.9661]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 236 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661],
                      [ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661],
                      [ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661],
                      [ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661],
                      [ 1.1137,  0.2588,  0.5976, -0.8652],
                      [-0.6017,  1.1726, -1.1364,  0.7534],
                      [ 0.8182, -1.7573,  0.1515, -0.9661],
                      [ 1.1137,  0.2588,  0.5976, -0.8652],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 2.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
